# Query EarthScope MT Data Holdings

In [1]:
import pathlib

In [2]:
doQuery = True
doSummary = True
doCharts = True
include_restricted = True


out_folder = pathlib.Path(".")
out_folder = out_folder.joinpath("data_availability")
if include_restricted:
    out_folder = out_folder.joinpath("restricted")
else:
    out_folder = out_folder.joinpath("public") #technically this is public & restricted
out_folder.mkdir(exist_ok=True, parents=True)
print(out_folder.absolute())

filebase = "mt_availability"
outfile = out_folder.joinpath(f"{filebase}.txt")

/home/kkappler/software/irismt/aurora/tests/earthscope/data_availability/restricted


In [3]:
import os
import pandas as pd
import pathlib
import time

fdsn_URL = "http://service.iris.edu/fdsnws"
channels=['?FE', '?FN', '?FZ','?F1','?F2', '?QE', '?QN', '?QZ', '?Q1', '?Q2']
channels = ','.join(channels)

if doQuery:
    with open(outfile, 'w') as f:
        f.write("net.sta,chan,hours")
        
    sta_URL=f"{fdsn_URL}/station/1/query?cha={channels}&level=channel&format=text&includecomments=true&nodata=204"
    print(sta_URL)

    try:
        network_df=pd.read_csv(sta_URL, sep='|')
    except Exception as e:
        print(f"ERROR with station service {sta_URL}")
        print(f"ERROR: {e}")
        quit()

    network_df
    networks=network_df['#Network '].unique()
    print(f"Identified {len(networks)} unique networks: \n {networks}")
    # grouped = stations.groupby(by=['#Network ', ' Station '])
    # grouped = network_df.groupby(by='#Network ')

    for network in networks:
        print(network)
        netfile = f'{filebase}_{network}.txt'
        netfile = out_folder.joinpath(netfile)
        
        av_URL = f"{fdsn_URL}/availability/1/query?format=text&net={network}&cha={channels}&orderby=nslc_time_quality_samplerate&includerestricted={include_restricted}&nodata=204" 

        print(av_URL)
        try:
            avail = pd.read_csv(av_URL, sep=" ")
        except Exception as e:
            print(f"ERROR with availability service {av_URL} ")
            print(f"ERROR: {e}")
            with open(outfile, 'a') as f:
                f.write(f"\n#ERROR with {network}")
            time.sleep(2)
            continue


        avail.columns = avail.columns.str.strip()
        avail['Latest'] = pd.to_datetime(avail['Latest'], format="%Y-%m-%dT%H:%M:%S.%f") 
        avail['Earliest'] = pd.to_datetime(avail['Earliest'],  format="%Y-%m-%dT%H:%M:%S.%f") 
        avail['Span'] = avail.Latest - avail.Earliest

        avail.to_csv(netfile, index=False)
        
        grouped_chan = avail.groupby(by=['Station','Channel'])
        for name, group in grouped_chan:
            station = name[0]
            channel = name[1]
            total_time = group['Span'].sum()
            with open(outfile, 'a') as f:
                f.write(f"\n{network}.{station},{channel},{'%.2f' % (total_time/pd.Timedelta(hours=1))}")
#                 f.write(f"\n{network}.{station},{channel},{total_time}")
                
        time.sleep(5)

http://service.iris.edu/fdsnws/station/1/query?cha=?FE,?FN,?FZ,?F1,?F2,?QE,?QN,?QZ,?Q1,?Q2&level=channel&format=text&includecomments=true&nodata=204
Identified 20 unique networks: 
 ['1H' '4P' '7I' '8J' '8P' 'AV' 'BK' 'EM' 'II' 'IU' 'N4' 'NV' 'SF' 'SN'
 'US' 'XB' 'XC' 'YB' 'Z7' 'ZU']
1H
http://service.iris.edu/fdsnws/availability/1/query?format=text&net=1H&cha=?FE,?FN,?FZ,?F1,?F2,?QE,?QN,?QZ,?Q1,?Q2&orderby=nslc_time_quality_samplerate&includerestricted=True&nodata=204
4P
http://service.iris.edu/fdsnws/availability/1/query?format=text&net=4P&cha=?FE,?FN,?FZ,?F1,?F2,?QE,?QN,?QZ,?Q1,?Q2&orderby=nslc_time_quality_samplerate&includerestricted=True&nodata=204
7I
http://service.iris.edu/fdsnws/availability/1/query?format=text&net=7I&cha=?FE,?FN,?FZ,?F1,?F2,?QE,?QN,?QZ,?Q1,?Q2&orderby=nslc_time_quality_samplerate&includerestricted=True&nodata=204
8J
http://service.iris.edu/fdsnws/availability/1/query?format=text&net=8J&cha=?FE,?FN,?FZ,?F1,?F2,?QE,?QN,?QZ,?Q1,?Q2&orderby=nslc_time_quality_samp

In [4]:
if doSummary:
    
    summaryDF = pd.DataFrame(columns=['Network','Channels', '# Stations','Average Time', 'Total Time'])    
    sumDF = pd.read_csv(outfile)
    sumDF[['net','sta']] = sumDF['net.sta'].str.split('.',expand=True)

    zDF = sumDF[sumDF.chan.str.endswith('Z', na=False)]


    grouped_sum = zDF.groupby(by=['net'])
    
    for name, group in grouped_sum:
        nsta = group.nunique(axis=0)['sta']
        avgTime = group['hours'].mean()
        totalTime = group['hours'].sum()
        chans = sumDF[sumDF['net']==name]['chan'].unique()
        newRow = [name, chans, nsta, avgTime, totalTime]
        
        summaryDF.loc[len(summaryDF.index)] = newRow
        

           
    summaryDF['Average Time'] = summaryDF['Average Time'].round().apply(pd.to_timedelta, unit='H')
    summaryDF['Total Time'] = summaryDF['Total Time'].round().apply(pd.to_timedelta, unit='H')
    
    display(summaryDF)
    

,Network,Channels,# Stations,Average Time,Total Time
0,1H,"[LFE, LFN, LFZ, LQE, LQN]",117,19 days 09:00:00,2267 days 01:00:00
1,4P,"[LFE, LFN, LFZ, LQE, LQN]",978,22 days 21:00:00,22382 days 20:00:00
2,8J,"[LFE, LFN, LFZ, LQE, LQN]",25,79 days 04:00:00,1979 days 01:00:00
3,8P,"[LFE, LFN, LFZ, LQE, LQN]",119,25 days 03:00:00,2989 days 07:00:00
4,AV,"[LFE, LFN, LFZ]",1,164 days 01:00:00,164 days 01:00:00
5,EM,"[LFE, LFN, LFZ, LQE, LQN, VFE, VFN, VFZ, VQE, ...",1728,24 days 05:00:00,41841 days 02:00:00
6,II,"[LFE, LFN, LFZ, BF1, BF2, BFE, BFN, BFZ, LF1, ...",2,4290 days 20:00:00,12872 days 12:00:00
7,IU,"[LF1, LF2, LFZ, UFZ, VFZ, LFE, LFN]",12,2978 days 10:00:00,41697 days 19:00:00
8,N4,"[LF1, LF2, LFZ]",2,975 days 17:00:00,1951 days 10:00:00
9,NV,"[MF1, MF2, MFZ, MFE, MFN]",2,261 days 01:00:00,522 days 03:00:00


In [5]:
if doCharts:
    from bokeh.plotting import figure, output_file, show, save
    from bokeh.layouts import column, gridplot
    from bokeh.models import ColumnDataSource, HoverTool
    from bokeh.models import Range1d
    from bokeh.io import output_notebook
    
    import os
    import fnmatch
    
    
    # Get list of all networks that have the availability files
    files = fnmatch.filter(os.listdir('.'), f'{filebase}_*.txt')
    
    
    
    s = list()
    idx = 0
    for file in files:
        network = file.split('_')[2].split('.')[0]
        print(network)
        
        # Timeline with number of stations depicted by width of line? 
        # Or two lines that have shaded area between?
        thisAvail = pd.read_csv(file,parse_dates=['Earliest','Latest'])
        earliest = min(thisAvail['Earliest'])
        latest = max(thisAvail['Latest'])
        print(earliest, latest)

        
        # create 100 bins for that timeframe(?)
        nbins = 100
        datelist = pd.date_range(earliest, latest, periods=100).tolist()

        numDF = pd.DataFrame(columns=['Date','Lower','Upper'])    

        for date in datelist:
            nsta = len(thisAvail[(thisAvail['Earliest']<=date) & (thisAvail['Latest']>=date)].Station.unique())
            numDF.loc[len(numDF.index)] = [date, -nsta, nsta]
            
        
        output_notebook()
        source = ColumnDataSource(numDF)
        
        if idx > 0:
            tmp_s = figure(width=800, height=100,x_axis_type="datetime",x_range=s[0].x_range)
        else: 
            tmp_s = figure(width=800, height=100,x_axis_type="datetime")
        s.append(tmp_s)
        
        s[idx].varea(x='Date', y1='Upper', y2='Lower', source=source)
        s[idx].y_range = Range1d(-30, 30)
        s[idx].title = network
        s[idx].add_tools(HoverTool(
            tooltips=[
                ( 'count', '@Upper'),
                ( 'time', '@Date'),
            ]))
        idx+=1
    
    p = gridplot(s, ncols=1)
    show(p)
    plot_file_name = out_folder.joinpath("mt_numbers.html")
    print(plot_file_name)
    output_file(plot_file_name)
    save(p)
    


        
        
    
        
        # Histogram of length of each station
        # Histogram of length of each segment? Do I have that info?



ModuleNotFoundError: No module named 'bokeh'